In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
import numpy as np
from io import StringIO 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from calendar import weekday

In [ ]:
def epf_arx(data, Ndays, startd, endd, forecast_type='naive'):
    first_day = str(int(data[0, 0]))
    first_day = (int(e) for e in (first_day[:4], first_day[4:6], first_day[6:]))
    i = weekday(*first_day) # Weekday of starting day: 0 - Monday, ..., 6 - Sunday
    N = len(data) // 24
    data = np.hstack([data, np.zeros((N*24, 9))]) # Append 'data' matrix with daily dummies & p_min
    for j in range(N):
        if i % 7 == 5:
            data[24*j:24*(j+1), 4] = 1 # Saturday dummy in 5th (index 4) column
        elif i % 7 == 6:
            data[24*j:24*(j+1), 5] = 1 # Sunday dummy in 6th column
        elif i % 7 == 0:
            data[24*j:24*(j+1), 6] = 1 # Monday dummy in 7th column
        elif i % 7 == 1:
            data[24*j:24*(j+1), 7] = 1 # wtorek
        elif i % 7 == 2:
            data[24*j:24*(j+1), 8] = 1 # sroda
        elif i % 7 == 3:
            data[24*j:24*(j+1), 9] = 1 # czwartek
        elif i % 7 == 4:
            data[24*j:24*(j+1), 10] = 1 # piatek

        i += 1
        data[24*j:24*(j+1), 11] = np.min(data[24*j:24*(j+1), 2]) # p_min in 8th column
    result = np.zeros((Ndays * 24, 4)) # Initialize `result` matrix
    result[:, :3] = data[endd*24:(endd + Ndays) * 24, :3]
    for j in range(Ndays):     # For all days ...
        for hour in range(24): # ... compute 1-day ahead forecasts for each hour
            data_h = data[hour::24, :]
            # Compute forecasts for the hour
            if forecast_type.lower() == 'arx':
                result[j * 24 + hour, 3] = forecast_arx(data_h[startd + j:endd + j + 1, :])


    return result

def forecast_arx(DATA):
    # DATA: 8-column matrix (date, hour, price, load forecast, Sat, Sun, Mon dummy, p_min)
    # Select data to be used
    # print(DATA[-1, :])
    price = DATA[:-1, 2]             # For day d (d-1, ...)
    price_min = DATA[:-1, 11]         # For day d
    Dummies = DATA[1:, 4:11]          # Dummies for day d+1
    loadr = DATA[1:, 3]              # Load for day d+1

    # Take logarithms
    price = np.log(price)
    mc = np.mean(price)
    price -= mc                      # Remove mean(price)
    price_min = np.log(price_min)
    price_min -= np.mean(price_min)  # Remove mean(price)
    loadr = np.log(loadr)

    # Calibrate the ARX model
    y = price[7:]                    # For day d, d-1, ...
    # Define explanatory variables for calibration
    # without intercept
    X = np.vstack([price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
                   loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2], Dummies[6:-1, 3], Dummies[6:-1, 4], Dummies[6:-1, 5], Dummies[6:-1, 6]]).T
    # with intercept
    # X = np.vstack([np.ones(len(y)), price[6:-1], price[5:-2], price[:-7], price_min[6:-1],
    #                loadr[6:-1], Dummies[6:-1, 0], Dummies[6:-1, 1], Dummies[6:-1, 2]]).T
    # Define explanatory variables for day d+1
    # without intercept
    X_fut = np.hstack([price[-1], price[-2], price[-7], price_min[-1],
                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2], Dummies[-1, 3], Dummies[-1, 4], Dummies[-1, 5], Dummies[-1, 6]])
    # with intercept
    # X_fut = np.hstack([[1], price[-1], price[-2], price[-7], price_min[-1],
    #                    loadr[-1], Dummies[-1, 0], Dummies[-1, 1], Dummies[-1, 2]])
    beta = np.linalg.lstsq(X, y, rcond=None)[0]  # Estimate the ARX model
    prog = np.dot(beta, X_fut)                   # Compute a step-ahead forecast
    return np.exp(prog + mc)                     # Convert to price level


In [ ]:
# Zadanie 1
data = np.loadtxt('GEFCOM.txt')

data = data[361*24:1082*24,:]

MAE_arr = []
RMSE_arr = []

for n in range(56,361):

    startd = n-56   # First day of the calibration window (startd from Matlab minus 1)
    endd = n   # First day to forecast (equal to endd from Matlab)
    Ndays = int(722/(360/n))  # user provided number of days to be predicted
                # (max is 722 for GEFCom with endd=360)

    # naive, arx, narx or narx_mc
    forecast_type = 'arx'

    # Estimate and compute forecasts of the ARX model
    res = epf_arx(data[:, :4], Ndays, startd, endd, forecast_type='arx')
    MAE_arr.append(np.mean(np.abs(res[:, 2] - res[:, 3])))
    RMSE_arr.append(np.power(np.mean(np.power(res[:, 2] - res[:, 3],2)),0.5))
    # print(n)

plt.scatter(MAE_arr,RMSE_arr)
plt.title(f"Wykres MAE i RMSE dla predykcji {Ndays} dni")
plt.xlabel("MAE")
plt.ylabel("RMSE")
plt.savefig('List_3v_zad_1.png', dpi=300)
plt.show()

In [ ]:
x_arr = np.linspace(56,360,305)

plt.plot(x_arr,MAE_arr)